In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import SRW_v041 as SRW
from scipy.sparse import csr_matrix, csc_matrix, issparse
import functools

# This toy example is for checking the partial derivatives

In [3]:
edges = np.array([[0,1],[0,2],[1,2],[1,0],[2,0],[2,1]])
features = csc_matrix([[0.9,0.1,1.],[0.1,0.9,1.],[0.5,0.5,1.],[0.9,0.1,1.],[0.1,0.9,1.],[0.5,0.5,1.]])
w0 = np.array([0.7,0.3,0.1])
w1 = np.array([0.700001,0.3,0.1])
rst_prob = 0.5
P_init = mutation_profile = csr_matrix([[1,0,0],[0,1,0],[0,1,1],[1,1,0],[1,1,1]])
nnodes = 3
group_labels = [0, 0, 1, 1, 1]
lam = 0.001

In [4]:
Q0, M_strength0, M_strength_rowSum0, strength_grad0 = SRW.strength_Q_and_gradient(edges, nnodes, features, w0)
Q1, M_strength1, M_strength_rowSum1, strength_grad1 = SRW.strength_Q_and_gradient(edges, nnodes, features, w1)

In [5]:
Q_grad = SRW.Q_gradient_1feature(edges, nnodes, M_strength0, M_strength_rowSum0, strength_grad0.toarray()[:,1]) 
Q_grad.toarray()

array([[ 0.        , -0.07991799,  0.07991799],
       [-0.03630052,  0.        ,  0.03630052],
       [ 0.04380975, -0.04380975,  0.        ]])

In [6]:
(Q1 - Q0).toarray() / 0.000001 # check

array([[ 0.        ,  0.061703  , -0.061703  ],
       [ 0.0273826 ,  0.        , -0.0273826 ],
       [-0.03446874,  0.03446874,  0.        ]])

In [7]:
P0 = csr_matrix(SRW.iterative_PPR(Q0.toarray(), SRW.renorm(mutation_profile).toarray(), rst_prob))
P1 = csr_matrix(SRW.iterative_PPR(Q1.toarray(), SRW.renorm(mutation_profile).toarray(), rst_prob))

In [8]:
P_grad = SRW.calc_P_grad_pool(edges, nnodes, M_strength0, M_strength_rowSum0, Q0, P0, rst_prob, strength_grad0)
P_grad[1,:,:]

array([[  9.28591017e-05,  -2.24112783e-02,   2.23184192e-02],
       [ -5.49892150e-03,  -9.83871874e-03,   1.53376402e-02],
       [  9.11765236e-04,  -1.32354706e-02,   1.23237053e-02],
       [ -2.70303120e-03,  -1.61249985e-02,   1.88280297e-02],
       [  6.38796524e-04,  -1.62940731e-02,   1.56552766e-02]])

In [9]:
(P1 - P0).toarray() / 0.000001 # check

array([[-0.00017466,  0.01735142, -0.01717676],
       [ 0.00403884,  0.00764556, -0.0116844 ],
       [-0.00091031,  0.01031926, -0.00940895],
       [ 0.00193209,  0.01249849, -0.01443058],
       [-0.00066509,  0.01266331, -0.01199822]])

In [10]:
SRW_obj_0 = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w=w0)
SRW_obj_1 = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w=w1)

In [11]:
J0, J_grad = SRW_obj_0.obj_func_and_grad(P0, P_grad, w0)
J1, J_grad1 = SRW_obj_1.obj_func_and_grad(P1, P_grad, w1)

In [12]:
J_grad

array([-0.00818094,  0.01312818,  0.00314724])

In [13]:
(J1 - J0) / 0.000001  # check

-0.0081796094897335081

In [14]:
SRW_obj_0.loss = SRW_obj_1.loss = 'silhouette'
SRW_obj_0.norm_type = SRW_obj_1.norm_type = 'L2'
J0, J_grad = SRW_obj_0.obj_func_and_grad(P0, P_grad, w0)
J1, J_grad1 = SRW_obj_1.obj_func_and_grad(P1, P_grad, w1)

In [15]:
J_grad

array([-0.02352117,  0.03336324,  0.00804207])

In [16]:
(J1 - J0) / 0.000001  # check

-0.023519133640625967

# This toy example is for checking the gradient descent functions

In [50]:
edges = [[0,1],[0,2],[0,3],[0,4],[0,5],[1,2],[1,3],[1,4],[1,5],[2,3],[2,4],[2,5],[3,4],[3,5],[4,5],
         [1,0],[2,0],[3,0],[4,0],[5,0],[2,1],[3,1],[4,1],[5,1],[3,2],[4,2],[5,2],[4,3],[5,3],[5,4],
         [0,0],[1,1],[2,2],[3,3],[4,4],[5,5]]
edges = np.array(edges)
features = csc_matrix([[.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                       [.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                       [.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.]])
rst_prob = 0.3
P_init = mutation_profile = csr_matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],
                                        [0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]).astype(float)
nnodes = 6
group_labels = [0, 0, 0, 1, 1, 1]
lam = 0.2
feature_names = ['f1', 'f2', 'selfloop', 'intercept']
node_names = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6']
sample_names = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [51]:
SRW_obj_2 = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w_init_sd=0.01, 
                           w=None, feature_names=feature_names, sample_names=sample_names, 
                           node_names=node_names, loss='silhouette', norm_type='L1', maximize_diff=True, 
                           learning_rate=0.1, update_w_func='Adam', P_init_val=P_init, 
                           group_labels_val=group_labels, eval_sil=True, ncpus=-1, maxit=1000, early_stop=5)

In [52]:
SRW_obj_2.train_SRW_GD()

finished calculating strength_grad: 12:55:00
finished network propagation: 12:55:00
finished calculating P_grad using pool: 12:55:00
finished calculating J and J_grad: 12:55:00
*** 0 iteration: J is -2.99818952991 cost_val is -2.99977736685
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.499962894475 silhouette_val is 0.499962894475 

finished calculating strength_grad: 12:55:00
finished network propagation: 12:55:00
finished calculating P_grad using pool: 12:55:01
finished calculating J and J_grad: 12:55:01
*** 1 iteration: J is -2.96148152712 cost_val is -3.03989368685
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.506648947809 silhouette_val is 0.506648947809 

finished calculating strength_grad: 12:55:01
finished network propagation: 12:55:01
finished calculating P_grad using pool: 12:55:01
finished calculating J and J_grad: 12:55:01
*** 2 iteration: J is -2.98167955441 cost_val is -3.07364325287
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.5

finished calculating P_grad using pool: 12:55:04
finished calculating J and J_grad: 12:55:04
*** 24 iteration: J is -3.17867768179 cost_val is -3.55748468991
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.592914114985 silhouette_val is 0.592914114985 

finished calculating strength_grad: 12:55:04
finished network propagation: 12:55:04
finished calculating P_grad using pool: 12:55:04
finished calculating J and J_grad: 12:55:04
*** 25 iteration: J is -3.18942663324 cost_val is -3.5865904038
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.5977650673 silhouette_val is 0.5977650673 

finished calculating strength_grad: 12:55:04
finished network propagation: 12:55:04
finished calculating P_grad using pool: 12:55:04
finished calculating J and J_grad: 12:55:04
*** 26 iteration: J is -3.19859991777 cost_val is -3.61817071718
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.603028452863 silhouette_val is 0.603028452863 

finished calculating strength_grad: 12:

finished calculating P_grad using pool: 12:55:08
finished calculating J and J_grad: 12:55:08
*** 48 iteration: J is -3.8072497903 cost_val is -4.95651587977
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.826085979961 silhouette_val is 0.826085979961 

finished calculating strength_grad: 12:55:08
finished network propagation: 12:55:08
finished calculating P_grad using pool: 12:55:08
finished calculating J and J_grad: 12:55:08
*** 49 iteration: J is -3.82356990821 cost_val is -5.00878446834
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.834797411389 silhouette_val is 0.834797411389 

finished calculating strength_grad: 12:55:08
finished network propagation: 12:55:08
finished calculating P_grad using pool: 12:55:08
finished calculating J and J_grad: 12:55:08
*** 50 iteration: J is -3.83950781301 cost_val is -5.05596134079
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.842660223464 silhouette_val is 0.842660223464 

finished calculating strength_grad:

In [53]:
SRW_obj_2.generate_Q_and_P_fin()

In [54]:
SRW_obj_2.w

array([  3.79503603e+00,  -1.30820375e-03,   2.11304898e-02,
        -3.68612453e+00])

In [38]:
SRW_obj_2.train_SRW_BFGS()

finished calculating strength_grad: 23:20:00
finished network propagation: 23:20:00
finished calculating P_grad using pool: 23:20:00
finished calculating J and J_grad: 23:20:00
finished calculating strength_grad: 23:20:00
finished network propagation: 23:20:00
finished calculating P_grad using pool: 23:20:00
finished calculating J and J_grad: 23:20:00
finished calculating strength_grad: 23:20:00
finished network propagation: 23:20:00
finished calculating P_grad using pool: 23:20:01
finished calculating J and J_grad: 23:20:01
finished calculating strength_grad: 23:20:01
finished network propagation: 23:20:01
finished calculating P_grad using pool: 23:20:01
finished calculating J and J_grad: 23:20:01
finished calculating strength_grad: 23:20:01
finished network propagation: 23:20:01
finished calculating P_grad using pool: 23:20:01
finished calculating J and J_grad: 23:20:01
finished calculating strength_grad: 23:20:01
finished network propagation: 23:20:01
finished calculating P_grad usi

finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating strength_grad: 23:20:07
finished network propagation: 23:20:07
finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating strength_grad: 23:20:07
finished network propagation: 23:20:07
finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating strength_grad: 23:20:07
finished network propagation: 23:20:07
finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating strength_grad: 23:20:07
finished network propagation: 23:20:07
finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating strength_grad: 23:20:07
finished network propagation: 23:20:07
finished calculating P_grad using pool: 23:20:07
finished calculating J and J_grad: 23:20:07
finished calculating s

finished calculating P_grad using pool: 23:20:13
finished calculating J and J_grad: 23:20:13
finished calculating strength_grad: 23:20:13
finished network propagation: 23:20:13
finished calculating P_grad using pool: 23:20:13
finished calculating J and J_grad: 23:20:13
finished calculating strength_grad: 23:20:13
finished network propagation: 23:20:13
finished calculating P_grad using pool: 23:20:14
finished calculating J and J_grad: 23:20:14
finished calculating strength_grad: 23:20:14
finished network propagation: 23:20:14
finished calculating P_grad using pool: 23:20:14
finished calculating J and J_grad: 23:20:14
finished calculating strength_grad: 23:20:14
finished network propagation: 23:20:14
finished calculating P_grad using pool: 23:20:14
finished calculating J and J_grad: 23:20:14
finished calculating strength_grad: 23:20:14
finished network propagation: 23:20:14
finished calculating P_grad using pool: 23:20:14
finished calculating J and J_grad: 23:20:14
finished calculating s

In [39]:
SRW_obj_2.generate_Q_and_P_fin()

In [40]:
SRW_obj_2.w_map

,Weight
f1,8.072513e+00
f2,-4.365184e-07
selfloop,-1.235648e-11
intercept,-7.581087e+00


### Check the transition matrix Q and propagation scores P

In [54]:
SRW_obj_2.Q_fin_df

,n1,n2,n3,n4,n5,n6
n1,0.000064,0.672312,0.323085,0.002246,0.001409,0.000884
n2,0.583852,0.000056,0.412150,0.001951,0.001224,0.000768
n3,0.402707,0.591556,0.000080,0.002800,0.001756,0.001102
n4,0.002241,0.002241,0.002241,0.000064,0.670837,0.322376
n5,0.001224,0.001224,0.001224,0.584010,0.000056,0.412262
n6,0.001104,0.001104,0.001104,0.403656,0.592951,0.000080


In [42]:
SRW_obj_2.P_fin_df

,n1,n2,n3,n4,n5,n6
p1,0.478620,0.256060,0.256060,0.003086,0.003086,0.003086
p2,0.256060,0.478620,0.256060,0.003086,0.003086,0.003086
p3,0.256060,0.256060,0.478620,0.003086,0.003086,0.003086
p4,0.003086,0.003086,0.003086,0.478620,0.256060,0.256060
p5,0.003086,0.003086,0.003086,0.256060,0.478620,0.256060
p6,0.003086,0.003086,0.003086,0.256060,0.256060,0.478620


In [43]:
print '*** cost is', SRW_obj_2.cost, 'cost_val is', SRW_obj_2.cost_val
print '*** accuracy is', SRW_obj_2.accuracy, 'accuracy_val is', SRW_obj_2.accuracy_val
print '*** silhouette is', SRW_obj_2.cost_sil, 'silhouette_val is', SRW_obj_2.cost_sil_val, '\n'

*** cost is -1.21836325717 cost_val is -1.21836325717
*** accuracy is 1.0 accuracy_val is 1.0
*** silhouette is 0.951095041457 silhouette_val is 0.951095041457 



### Compare with unweighted random walk

In [396]:
SRW_obj_3 = SRW.SRW_solver(edges, features, nnodes, P_init, rst_prob, group_labels, lam, w_init_sd=0.01, 
                           w=[0.,0.,0.,0.], 
                           feature_names=feature_names, loss='squared', norm_type='L2', maximize_diff=False, 
                           learning_rate=1, update_w_func='Adam', P_init_val=P_init, 
                           group_labels_val=group_labels, eval_sil=True)
SRW_obj_3.generate_Q_and_P_fin()

In [397]:
SRW_obj_3.Q_fin.toarray()

array([[ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667]])

In [398]:
SRW_obj_3.P_fin

array([[ 0.41666667,  0.11666667,  0.11666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.41666667,  0.11666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.41666667,  0.11666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.41666667,  0.11666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.11666667,  0.41666667,
         0.11666667],
       [ 0.11666667,  0.11666667,  0.11666667,  0.11666667,  0.11666667,
         0.41666667]])

In [399]:
SRW_obj_3.calc_cost_val(SRW_obj_3.P_fin, SRW_obj_3.w)
SRW_obj_3.cost_val

0.36000000000000004

# Scratch

In [123]:
grouplabel2index = {}
for i, grouplabel in enumerate(sorted(list(set(group_labels)))):
    grouplabel2index[grouplabel] = i

ngroups = len(grouplabel2index)
group2indeces_list = [[] for _ in range(ngroups)]
for i in range(len(group_labels)):
    groupindex = grouplabel2index[group_labels[i]]
    group2indeces_list[groupindex].append(i)

In [163]:
# Define a matrix C (g by n), which contains the group centroid
C = np.zeros((ngroups, P.shape[1]))
# Define a matrix C_grad (w by g by n), which contains the gradient of group centroid
C_grad = np.zeros((P_grad.shape[0], ngroups, P_grad.shape[1]))
for i in range(ngroups):
    C[i,:] = P.take(group2indeces_list[i], axis=0).mean(axis=0)
    C_grad[:,i,:] = P_grad.take(group2indeces_list[i], axis=1).mean(axis=1)

In [157]:
P_grad.take(group2indeces_list[0], axis=1).mean(axis=1)

array([[  4.35882908e-04,   4.35882904e-04,   4.35882911e-04,
         -4.35882908e-04,  -4.35882908e-04,  -4.35882908e-04],
       [ -4.03898710e-04,  -6.12166192e-03,   5.31386450e-03,
          3.71003686e-04,   3.78563823e-04,   4.62128620e-04],
       [  3.54144122e-07,   3.54144120e-07,   3.54144124e-07,
         -3.54144122e-07,  -3.54144122e-07,  -3.54144122e-07],
       [ -8.07447885e-04,  -8.07447889e-04,  -8.07447881e-04,
          8.07447885e-04,   8.07447885e-04,   8.07447885e-04]])

In [283]:
SRW.obj_func_silhouette(P, P_grad, group_labels, lam, w, norm_type='L2')

(1.5126989336996985,
 array([  1.79649141e-01,  -3.26623413e-10,  -9.97514024e-09,
         -1.68030186e-01]),
 1.0)

In [278]:
SRW.eval_acc_wrapper(P, group_labels)

(0.0, 1.0)